In [104]:
import os
import re
import string
import polars as pl

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">
<h3 align="left"><font color='#DEB887'>Info</font></h3>
    
This notebook allows you to concatenate and filter the data necessary to train the Machine Learning model for the Recommendation System

# List all reviews parquet files related to reviews to be loaded

In [105]:
path = '../data/raw/reviews'
with os.scandir(path) as reviewFiles:
    reviewFiles = [file.name for file in reviewFiles if file.is_file()]
print(reviewFiles)

['processed_reviews_books_1.parquet', 'processed_reviews_books_10.parquet', 'processed_reviews_books_11.parquet', 'processed_reviews_books_12.parquet', 'processed_reviews_books_13.parquet', 'processed_reviews_books_2.parquet', 'processed_reviews_books_3.parquet', 'processed_reviews_books_4.parquet', 'processed_reviews_books_5.parquet', 'processed_reviews_books_6.parquet', 'processed_reviews_books_7.parquet', 'processed_reviews_books_8.parquet', 'processed_reviews_books_9.parquet']


# Load and join all parquet files in the directory

In [106]:
df = pl.DataFrame()
for f in reviewFiles:
    n = str.format("../data/raw/reviews/{}", f)
    print("reading " + n)

    df_aux = pl.read_parquet(n)

    print("Adding..." + n)
    df = df.vstack(df_aux)
    n = ""


reading ../data/raw/reviews/processed_reviews_books_1.parquet
Adding...../data/raw/reviews/processed_reviews_books_1.parquet
reading ../data/raw/reviews/processed_reviews_books_10.parquet
Adding...../data/raw/reviews/processed_reviews_books_10.parquet
reading ../data/raw/reviews/processed_reviews_books_11.parquet
Adding...../data/raw/reviews/processed_reviews_books_11.parquet
reading ../data/raw/reviews/processed_reviews_books_12.parquet
Adding...../data/raw/reviews/processed_reviews_books_12.parquet
reading ../data/raw/reviews/processed_reviews_books_13.parquet
Adding...../data/raw/reviews/processed_reviews_books_13.parquet
reading ../data/raw/reviews/processed_reviews_books_2.parquet
Adding...../data/raw/reviews/processed_reviews_books_2.parquet
reading ../data/raw/reviews/processed_reviews_books_3.parquet
Adding...../data/raw/reviews/processed_reviews_books_3.parquet
reading ../data/raw/reviews/processed_reviews_books_4.parquet
Adding...../data/raw/reviews/processed_reviews_books_4.

In [107]:
df.sample(1)

asin,overall,reviewText,reviewerID,summary,verified
str,str,str,str,str,str
"""0316512583""","""5.0""","""I just got thi…","""A1JKKOGNN489FS…","""I love this bo…","""true"""


In [108]:
df.shape

(2918585, 6)

In [109]:
df.null_count()

asin,overall,reviewText,reviewerID,summary,verified
u32,u32,u32,u32,u32,u32
0,0,1403,0,871,0


In [110]:
df = df.drop_nulls()

In [111]:
df.shape

(2916355, 6)

Save it as temporal parquet file

In [112]:
df.write_parquet("../data/processed/All_reviews.parquet")

In [113]:
reviews = pl.read_parquet("../data/processed/All_reviews.parquet")

Load the metadata related to books category

In [114]:
metadata = pl.read_parquet("../data/raw/all_metadata_books_in_top25k.parquet")

In [115]:
reviews.shape

(2916355, 6)

In [116]:
metadata.shape

(12049, 9)

In [117]:
reviews.sample(1)

asin,overall,reviewText,reviewerID,summary,verified
str,str,str,str,str,str
"""0316389706""","""4.0""","""I was introduc…","""A35X82YUOSSOCS…","""Good book, but…","""true"""


In [118]:
metadata.sample(1)

asin,brand,price,title,cant_image,cant_category,rank_in_category,also_buy_count,also_view_count
str,str,f64,str,u32,u32,i64,u32,u32
"""1456468693""","""Fred Mcallen""",13.46,"""Charting and T…",0,3,8617,99,60


Join 2 dataframes by the ASIN number

In [119]:
join_keys = ["asin"]
df_joined = metadata.join(reviews, join_keys)

In [120]:
df_joined.shape

(2916355, 14)

In [121]:
df_joined.sample(1)

asin,brand,price,title,cant_image,cant_category,rank_in_category,also_buy_count,also_view_count,overall,reviewText,reviewerID,summary,verified
str,str,f64,str,u32,u32,i64,u32,u32,str,str,str,str,str
"""1930084072""","""Visit Amazon's…",2.89,"""Catholic Verse…",0,0,20453,100,58,"""5.0""","""Very helpful w…","""AWGSJH8JYT8XP""","""Very helpful w…","""true"""


In [133]:
final = df_joined.select("title", "reviewerID")


In [134]:
final.sample(1)

title,reviewerID
str,str
"""The Letter""","""A2MTFEQ0M7E4XP…"


In [135]:
final.shape
#2916355

(2916355, 2)

In [136]:
final = final.unique()

In [137]:
final.shape

(2905278, 2)

Save the file to be used

In [138]:
final.write_parquet("../data/processed/Final_data_for_ML.parquet")